In [12]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
%%time
df_full = pd.read_csv('../data/processed/newsgroup_vectorized.gz', compression='gzip')

Wall time: 3min 43s


## Split dataset in train, test and validation

In [3]:
# %%time
# X = df_full.drop(columns={'target'})
# y = df_full[['target']]

# X, X_test, y, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

Wall time: 7.34 s


## Stratified K Fold Cross Validation

In [4]:
# %%time

# skf = StratifiedKFold(n_splits=5, random_state=42)

# list_metrics = []
# for train_index, val_index in skf.split(X, y):
#     X_train = X.iloc[train_index]
#     X_val = X.iloc[val_index]
#     y_train = y.iloc[train_index]['target']
#     y_val = y.iloc[val_index]['target']
    
#     model = MLPClassifier()
#     model.fit(X_train, y_train)
    
#     list_metrics.append(round(accuracy_score(y_val, model.predict(X_val)), 3))   

## Evaluate Metrics 

In [5]:
# list_metrics

In [6]:
# round(np.mean(list_metrics), 2)

## Validate Model with test set

In [7]:
# accuracy_score(y_test, model.predict(X_test))

## Train model

In [156]:
%%time
X = df_full.drop(columns={'target'})
y = df_full[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Wall time: 5.08 s


In [171]:
%%time
model = MLPClassifier()
model.fit(X_train, y_train)

C:\Users\filarf\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 24min 34s


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

## Error analysis

In [172]:
df_validation = pd.DataFrame()
df_validation['y'] = y_test['target']
df_validation['y_hat'] = model.predict(X_test)

df_tp = df_validation[df_validation['y'] == df_validation['y_hat']].groupby('y_hat').size()
df_fn = df_validation[df_validation['y'] != df_validation['y_hat']].groupby('y').size()
df_fp = df_validation[df_validation['y'] != df_validation['y_hat']].groupby('y_hat').size()

In [173]:
accuracy_score(y_test, model.predict(X_test))

0.8860860329261816

In [175]:
list_result = []
for target_class in df_validation['y'].unique():
    list_result.append({
        'class' : target_class,
        'precision' : round(df_tp.loc[target_class] / (df_tp.loc[target_class] + df_fp.loc[target_class]), 2),
        'recall' : round(df_tp.loc[target_class] / (df_tp.loc[target_class] + df_fn.loc[target_class]), 2),
    })

In [176]:
pd.DataFrame(list_result).sort_values('class')

,class,precision,recall
8,alt.atheism,0.97,0.87
10,comp.graphics,0.77,0.82
9,comp.os.ms-windows.misc,0.77,0.85
18,comp.sys.ibm.pc.hardware,0.77,0.75
1,comp.sys.mac.hardware,0.86,0.85
3,comp.windows.x,0.85,0.86
7,misc.forsale,0.77,0.85
0,rec.autos,0.93,0.89
15,rec.motorcycles,0.98,0.93
2,rec.sport.baseball,0.99,0.92
